In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
split_lines = [l.split("~") for l in lines]
bricks = [
    [(int(sl.split(",")[0]), int(sl.split(",")[1]), int(sl.split(",")[2])) for sl in l]
    for l in split_lines
]
bricks

In [ ]:
# 0 x, 1 y, 2 z
def get_min_coord(brick, dir):
    return min(brick[0][dir], brick[1][dir])


def get_max_coord(brick, dir):
    return max(brick[0][dir], brick[1][dir])


def get_min_z_coord(brick):
    return get_min_coord(brick, 2)


def brick_eq(brick1, brick2):
    for d in range(0, 3):
        if get_min_coord(brick1, d) != get_min_coord(brick2, d) or get_max_coord(
            brick1, d
        ) != get_max_coord(brick2, d):
            return False
    return True


def brick_intersect(brick1, brick2):
    dim_intersect_counts = 0
    for d in range(0, 3):
        if (
            get_min_coord(brick1, d) <= get_max_coord(brick2, d)
            and get_max_coord(brick1, d) >= get_min_coord(brick2, d)
        ) or (
            get_min_coord(brick2, d) <= get_max_coord(brick1, d)
            and get_max_coord(brick2, d) >= get_min_coord(brick1, d)
        ):
            dim_intersect_counts += 1

    return dim_intersect_counts == 3


def mv_pos(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1], pos[2] + dir[2])


def mv_brick(brick, dir):
    return [mv_pos(c, dir) for c in brick]


def try_brick_fall(brick, col_bricks):
    new_potential_brick = mv_brick(brick, (0, 0, -1))

    if get_min_z_coord(new_potential_brick) < 1:
        return brick

    # Ensure bricks are sorted by z coordinate asc, so
    # later bricks are potentially higher, so collide earlier -> reverse
    for cb in reversed(col_bricks):
        if brick_intersect(new_potential_brick, cb):
            return brick

    return new_potential_brick


# brick_intersect([(5,1,1),(4,1,1)],[(4,0,0),(4,0,0)])
# mv_brick([(1,1,1),(1,1,1)], (0,0,1))

In [ ]:
z_sorted_bricks = sorted(bricks, key=lambda b: get_min_z_coord(b))
z_sorted_bricks

In [ ]:
moved_bricks = []
curr_max_height = 1
for i in range(0, len(z_sorted_bricks)):
    new_brick = z_sorted_bricks[i]
    brick_still_moving = True

    # fast fall until collision possible
    brick_min_z = get_min_z_coord(new_brick)
    while brick_min_z > curr_max_height + 1:
        new_brick = mv_brick(new_brick, (0, 0, -1))
        brick_min_z = get_min_z_coord(new_brick)

    # if collision possible, check with all lower bricks, because they already fell as far as they could
    # Still super uneffective, but with about 6 seconds fast enough
    while brick_still_moving:
        fallen_brick = try_brick_fall(new_brick, moved_bricks[:i])
        if brick_eq(new_brick, fallen_brick):
            brick_still_moving = False
            curr_max_height = max(curr_max_height, get_max_coord(fallen_brick, 2))
            moved_bricks.append(fallen_brick)
        else:
            new_brick = fallen_brick

moved_bricks

In [ ]:
z_sorted_moved_bricks = sorted(moved_bricks, key=lambda b: get_min_z_coord(b))
z_sorted_moved_bricks

In [ ]:
# Give a name to every brick, this makes the later parts faster for dict lookups
named_moved_bricks = {tuple(v): k for k, v in enumerate(z_sorted_moved_bricks)}
named_moved_bricks

In [ ]:
# Check if the brick supports any other bricks
# If False, the brick is on top of its stack (may be a stack of 1 brick though)
def supports_another(brick, bricks):
    max_z = get_max_coord(brick, 2)
    potential_supported = [b for b in bricks if get_min_z_coord(b) == max_z + 1]
    for ps in potential_supported:
        if brick_intersect(mv_brick(brick, (0, 0, 1)), ps):
            return True
    return False


# Bricks that are on top and can be disintegrated
top_bricks = [
    b for b in z_sorted_moved_bricks if not supports_another(b, z_sorted_moved_bricks)
]
top_bricks

In [ ]:
# Compute a dict that maps each brick to a list of bricks it directly supports
support_dict = {}


def find_supported_bricks(brick, bricks):
    max_z = get_max_coord(brick, 2)
    potential_supported = [b for b in bricks if get_min_z_coord(b) == max_z + 1]
    for ps in potential_supported:
        if brick_intersect(mv_brick(brick, (0, 0, 1)), ps):
            if named_moved_bricks[tuple(brick)] not in support_dict:
                support_dict[named_moved_bricks[tuple(brick)]] = [
                    named_moved_bricks[tuple(ps)]
                ]
            else:
                support_dict[named_moved_bricks[tuple(brick)]].append(
                    named_moved_bricks[tuple(ps)]
                )


for b in z_sorted_moved_bricks:
    find_supported_bricks(b, z_sorted_moved_bricks)

support_dict

In [ ]:
# For each brick map it to a list of bricks it is supported by
# If this is empty, it would fall (because all bricks are already dropped as low as they can go)
supported_by_dict = {}
for k, v in support_dict.items():
    for brick in v:
        if brick not in supported_by_dict:
            supported_by_dict[brick] = [k]
        else:
            supported_by_dict[brick].append(k)
supported_by_dict

In [ ]:
# Get a list of bricks that are not the only support for any other brick
# This allows them to be disintegrated without something falling
not_single_support_bricks = []
for k in support_dict.keys():
    k_single_support_counts = 0
    for v in supported_by_dict.values():
        if len(v) == 1 and k in v:
            k_single_support_counts += 1
    if k_single_support_counts == 0:
        not_single_support_bricks.append(k)
not_single_support_bricks

In [ ]:
len(not_single_support_bricks) + len(top_bricks)

part 2

In [ ]:
# If a set of bricks is falling, what other bricks are now unsupported (and will consequently fall too)
def get_now_unsupported_bricks(falling_bricks, support_dict, supported_by_dict):
    supported_bricks = []
    for b in falling_bricks:
        supported_bricks += support_dict.get(b, [])

    now_unsupported = []
    for sp in supported_bricks:
        if len(set(supported_by_dict[sp]).difference(set(falling_bricks))) == 0:
            now_unsupported.append(sp)

    return list(set(now_unsupported))


# If a single brick falls or disappears, how many other bricks are also falling then
def get_falling_bricks(brick, support_dict, supported_by_dict):
    falling_bricks_count = 0
    falling_bricks = [brick]

    still_more_falling = True
    # Add newly falling bricks until no more bricks are falling
    while still_more_falling:
        falling_bricks = list(
            set(
                falling_bricks
                + get_now_unsupported_bricks(
                    falling_bricks, support_dict, supported_by_dict
                )
            )
        )
        if len(falling_bricks) > falling_bricks_count:
            falling_bricks_count = len(falling_bricks)
        else:
            still_more_falling = False
    # The initial brick isn't counted
    return falling_bricks_count - 1


# get_now_unsupported_bricks([6], support_dict, supported_by_dict)
get_falling_bricks(5, support_dict, supported_by_dict)

In [ ]:
fb = 0
for i in range(0, len(z_sorted_moved_bricks)):
    fb += get_falling_bricks(i, support_dict, supported_by_dict)
fb